In [1]:
import transformers

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
tf.__version__

'2.3.0'

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  print("Invalid device or cannot modify virtual devices once initialized.")

In [4]:
train_df = pd.read_json('data/train.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]

In [5]:
val_df = pd.read_json('data/dev.jsonl', lines = True)
val_df["img"] = "data/" + val_df["img"]

In [6]:
train_df = pd.concat([train_df, val_df])

In [7]:
img_width = 128
img_height = 128

In [8]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [9]:
max_len = 128

### Train

In [10]:
def build_model():
    
    encoder = transformers.TFDistilBertModel.from_pretrained("distilbert-base-uncased")
    encoder.trainable = False

    input_img = tf.keras.layers.Input(
        shape = (img_width, img_height, 3), name = "image"
    )
    
    input_ids = tf.keras.layers.Input(name = "input_ids", shape = (max_len,), dtype = tf.int32)

    attention_mask = tf.keras.layers.Input(name = "attention_mask", shape = (max_len,), dtype = tf.int32)

    y = encoder(input_ids, attention_mask = attention_mask)[0]

    extractor = tf.keras.applications.EfficientNetB0(include_top = False, \
                                                 input_tensor = input_img, weights = "imagenet")
    
    extractor.trainable = False
    
    for layer in extractor.layers:
        if (layer.name.startswith("block7") or layer.name.startswith("block6")):
            layer.trainable = True

    x = tf.keras.layers.GlobalMaxPooling2D()(extractor.output)

    y = tf.keras.layers.GlobalMaxPooling1D()(y)
    
    x = tf.keras.layers.concatenate([x, y])
    
    x = tf.keras.layers.Dense(1024)(x)
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    x = tf.keras.layers.Dense(128)(x)
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    out = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = tf.keras.models.Model([input_img, input_ids, attention_mask], out)

    return model

In [11]:
model = tf.keras.model.load_model("hateful_flatten")
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model.summary()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 128, 128, 3)  0           image[0][0]                      
__________________________________________________________________________________________________
normalization (Normalization)   (None, 128, 128, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 129, 129, 3)  0           normalization[0][0]              
_______________________________________________________________________________________

In [12]:
def encode_single_sample(img_path, label, text):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.int32)
    img = tf.image.resize(img, [img_height, img_width])
    text = tokenizer(text, return_tensors = 'tf')
    return {"image": img, "label": label, "text": text}

In [13]:
class HatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, label, text):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.text = text
        self.label = label

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_label = self.label[i : i + self.batch_size]
        batch_text = self.text[i : i + self.batch_size]
        w = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        x = np.zeros((self.batch_size,max_len,), dtype = "int32")
        y = np.zeros((self.batch_size,max_len,), dtype = "int32")
        z = np.zeros((self.batch_size), dtype = "float32")
        
        for i,j,k,l in zip(range(self.batch_size), batch_input_img_paths, batch_label, batch_text):
            sample = encode_single_sample(j,k,l)
            w[i] = sample["image"].numpy().tolist()
            input_ids = sample["text"]['input_ids'].numpy().tolist()[0]
            for _ in range(max_len - len(input_ids)):
              input_ids.append(0)
            x[i] = input_ids
            attention_mask = sample["text"]['attention_mask'].numpy().tolist()[0]
            for _ in range(max_len - len(attention_mask)):
              attention_mask.append(0)
            y[i] = attention_mask
            z[i] = sample["label"]
        
        return [w,x,y], z

In [14]:
train_gen = HatefulMemes(16, train_df["img"].values.tolist(), train_df["label"].values.tolist(), \
                         train_df["text"].values.tolist())

In [17]:
epochs = 20
early_stopping_patience = 3

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = "val_accuracy", patience = early_stopping_patience, restore_best_weights = True
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'hateful_flatten', monitor = 'val_accuracy', verbose = 1, save_best_only = True, mode = 'max'
)

history = model.fit(train_gen, validation_data = val_gen, epochs = epochs, \
                    callbacks = [checkpoint])

Epoch 1/20
468/468 [==============================] - ETA: 0s - loss: 0.5283 - accuracy: 0.7376
Epoch 00001: val_accuracy improved from -inf to 0.62500, saving model to hateful_flatten
INFO:tensorflow:Assets written to: hateful_flatten/assets
468/468 [==============================] - 504s 1s/step - loss: 0.5283 - accuracy: 0.7376 - val_loss: 0.7947 - val_accuracy: 0.6250
Epoch 2/20
468/468 [==============================] - ETA: 0s - loss: 0.5220 - accuracy: 0.7428
Epoch 00002: val_accuracy improved from 0.62500 to 0.63844, saving model to hateful_flatten
INFO:tensorflow:Assets written to: hateful_flatten/assets
468/468 [==============================] - 504s 1s/step - loss: 0.5220 - accuracy: 0.7428 - val_loss: 0.7233 - val_accuracy: 0.6384
Epoch 3/20
468/468 [==============================] - ETA: 0s - loss: 0.5238 - accuracy: 0.7399
Epoch 00003: val_accuracy did not improve from 0.63844
468/468 [==============================] - 465s 994ms/step - loss: 0.5238 - accuracy: 0.7399 - v

468/468 [==============================] - ETA: 0s - loss: 0.4681 - accuracy: 0.7766
Epoch 00015: val_accuracy did not improve from 0.64852
468/468 [==============================] - 437s 934ms/step - loss: 0.4681 - accuracy: 0.7766 - val_loss: 0.9248 - val_accuracy: 0.5981
Epoch 16/20
468/468 [==============================] - ETA: 0s - loss: 0.4667 - accuracy: 0.7775
Epoch 00016: val_accuracy did not improve from 0.64852
468/468 [==============================] - 438s 936ms/step - loss: 0.4667 - accuracy: 0.7775 - val_loss: 0.9768 - val_accuracy: 0.5659
Epoch 17/20
468/468 [==============================] - ETA: 0s - loss: 0.4667 - accuracy: 0.7767
Epoch 00017: val_accuracy did not improve from 0.64852
468/468 [==============================] - 437s 933ms/step - loss: 0.4667 - accuracy: 0.7767 - val_loss: 0.8086 - val_accuracy: 0.5867
Epoch 18/20
468/468 [==============================] - ETA: 0s - loss: 0.4595 - accuracy: 0.7834
Epoch 00018: val_accuracy did not improve from 0.64852

### Predict

In [59]:
train_df = pd.read_json('data/test.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]

In [60]:
train_df

,id,img,text
0,16395,data/img/16395.png,handjobs sold seperately
1,37405,data/img/37405.png,introducing fidget spinner for women
2,94180,data/img/94180.png,happy pride month let's go beat up lesbians
3,54321,data/img/54321.png,laughs in [majority of u.s crime rate]
4,97015,data/img/97015.png,finds out those 72 virgins.. are goats
...,...,...,...
995,3869,data/img/03869.png,a mother's love for the child is a divine thing
996,23817,data/img/23817.png,sea monkeys
997,56280,data/img/56280.png,little miss muffet sat on her tuffet
998,29384,data/img/29384.png,they're in a row


In [61]:
def val_encode_single_sample(img_path, text):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.int32)
    img = tf.image.resize(img, [img_height, img_width])
    text = tokenizer(text, return_tensors = 'tf')
    return {"image": img, "text": text}

In [62]:
class ValHatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, text):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.text = text

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_text = self.text[i : i + self.batch_size]
        w = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        x = np.zeros((self.batch_size,max_len,), dtype = "int32")
        y = np.zeros((self.batch_size,max_len,), dtype = "int32")
        
        for i,j,k in zip(range(self.batch_size), batch_input_img_paths, batch_text):
            sample = val_encode_single_sample(j,k)
            w[i] = sample["image"].numpy().tolist()
            input_ids = sample["text"]['input_ids'].numpy().tolist()[0]
            for _ in range(max_len - len(input_ids)):
              input_ids.append(0)
            x[i] = input_ids
            attention_mask = sample["text"]['attention_mask'].numpy().tolist()[0]
            for _ in range(max_len - len(attention_mask)):
              attention_mask.append(0)
            y[i] = attention_mask
        
        return [w,x,y]

In [63]:
pred_gen = ValHatefulMemes(1, train_df["img"].values.tolist(), train_df["text"].values.tolist())

In [64]:
preds = model.predict(pred_gen, verbose = 1)

1000/1000 [==============================] - 63s 63ms/step


In [65]:
def prob2pred(x):
    if x > 0.5:
        return 1
    else:
        return 0

In [67]:
train_df["proba"] = [i[0] for i in preds.tolist()]
train_df["label"] = train_df["proba"].apply(prob2pred)

In [68]:
train_df = train_df.drop(["img","text"], axis = 1)
train_df

,id,proba,label
0,16395,0.010531,0
1,37405,0.353186,0
2,94180,0.374418,0
3,54321,0.470910,0
4,97015,0.161655,0
...,...,...,...
995,3869,0.427816,0
996,23817,0.117368,0
997,56280,0.322396,0
998,29384,0.030499,0


In [69]:
train_df.to_csv("submission.csv", index = False)